# Exact NEPEC representations vs exact PEC representations

One of the key steps for implementing probabilistic error cancellation is to represent each ideal gate of a circuit of interest as a linear combination of a set of noisy operations which are implementable on hardware.

In this notebook we compare the optimal representation of a an ideal gate obtained with:
- a fixed-noise basis of implementable operations (PEC)
- a noise-scaled basis of implementable operations (NEPEC).

We assume an amplitude damping noise model.

## Imports

In [9]:
import numpy as np
from cirq import LineQubit, channel, Circuit, X, Y, Z, I, H, CNOT, S, reset, T

from mitiq.pec.types import NoisyOperation, NoisyBasis, OperationRepresentation
from mitiq.pec.representations.damping import amplitude_damping_kraus
from mitiq.pec.channels import kraus_to_super
from mitiq.pec.representations.optimal import find_optimal_representation


## Define the ideal operation that we aim to represent

In [10]:
q = LineQubit(0)
ideal_operation = Circuit(I(q))

## Define an ideal basis (gate set)
We consider 3 different get sets. The results will be different depending on the choice of the gate set.

- If RESET is in the gate set, NEPEC is not beneficial (same sampling cost of PEC).
- If RESET is missing in the gate set PEC fails to represent the ideal gate, while NEPEC succeeds. 
- Moreover NEPEC can be applied even with a single gate and 3 scale factors (gate extrapolation method).

In [11]:
# basis_gates = [I, Z, reset]
basis_gates = [I, Z]
# basis_gates = [I]

## Define noisy basis of implementable operations (for PEC)

In [4]:
# Base noise of the quantum device
damp_level = 0.1

damp_basis_elements = [
    NoisyOperation(
        circuit=Circuit(gate(q)),
        channel_matrix=
            kraus_to_super(amplitude_damping_kraus(damp_level, 1)) @
            kraus_to_super(channel(gate(q))),
    )
    for j, gate in enumerate(basis_gates)
]

damp_basis = NoisyBasis(*damp_basis_elements)
print("Number of implementable operations in the PEC basis:", len(damp_basis_elements))

Number of implementable operations in the PEC basis: 2


## Define noisy basis of noise-scaled implementable operations (for NEPEC)

In [5]:
# Noise scale factors to be used for defining the NEPEC basis
scale_factors = [1, 4, 8]

if max(scale_factors) * damp_level > 1:
    damp_basis_scaled = None
    raise ValueError("scalfe_factor * damp_level cannot be larger than 1")

damp_basis_elements_scaled = []

for scale_factor in scale_factors:
    damp_basis_elements_scaled += [
        NoisyOperation(
            circuit=Circuit(gate(q)),
            channel_matrix=
            kraus_to_super(amplitude_damping_kraus(scale_factor * damp_level, 1)) @ 
            kraus_to_super(channel(gate(q))),
        )
        for j, gate in enumerate(basis_gates)
    ]

damp_basis_scaled = NoisyBasis(*damp_basis_elements_scaled)
print("Number of implementable operations in the NEPEC basis:", len(damp_basis_elements_scaled))

Number of implementable operations in the NEPEC basis: 6


## Find the PEC representation with minimum sampling cost (minimum one-norm)

In [6]:
try:
    damp_rep = find_optimal_representation(
        ideal_operation,
        noisy_basis=damp_basis,
    )
    pec_failed = False
    print("Optimal PEC representation:", damp_rep)
    print("One-norm of PEC representation:", damp_rep.norm)
except RuntimeError:
    pec_failed = True
    print("Optimization for PEC representation failed.")

Optimization for PEC representation failed.


## Find the PEC representation with minimum sampling cost (minimum one-norm)optimal representation

In [7]:
try:
    damp_rep_scaled = find_optimal_representation(
        ideal_operation,
        noisy_basis=damp_basis_scaled,
    )
    nepec_failed = False
except (RuntimeError, AttributeError):
    nepec_failed = True
    nepec_failed = True


if pec_failed:
    print("Optimization for PEC representation failed.")
else:
    print("One-norm of PEC representation:  ", damp_rep.norm)

if nepec_failed:
    print("Optimization for NEPEC representation failed.")
else:
    print("One-norm of NEPEC representation:", damp_rep_scaled.norm)
   

Optimization for PEC representation failed.
One-norm of NEPEC representation: 1.2857150682209122


### Hint: 
Go back to the section **Define an ideal basis**, choose a different gate set, and re-execute the notebook.

In [8]:
import mitiq

mitiq.about()


Mitiq: A Python toolkit for implementing error mitigation on quantum computers
Authored by: Mitiq team, 2020 & later (https://github.com/unitaryfund/mitiq)

Mitiq Version:	0.10.0dev

Core Dependencies
-----------------
Cirq Version:	0.10.0
NumPy Version:	1.20.1
SciPy Version:	1.4.1

Optional Dependencies
---------------------
PyQuil Version:	2.28.0
Qiskit Version:	0.24.0
Braket Version:	1.5.16

Python Version:	3.7.7
Platform Info:	Linux (x86_64)
